In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ai-of-god-v20/sample_submission.csv
/kaggle/input/ai-of-god-v20/train.csv
/kaggle/input/ai-of-god-v20/test.csv
/kaggle/input/ai-of-god-v20/test/1269.jpg
/kaggle/input/ai-of-god-v20/test/623.jpg
/kaggle/input/ai-of-god-v20/test/2193.jpg
/kaggle/input/ai-of-god-v20/test/2008.jpg
/kaggle/input/ai-of-god-v20/test/2081.jpg
/kaggle/input/ai-of-god-v20/test/764.jpg
/kaggle/input/ai-of-god-v20/test/1700.jpg
/kaggle/input/ai-of-god-v20/test/1786.jpg
/kaggle/input/ai-of-god-v20/test/1075.jpg
/kaggle/input/ai-of-god-v20/test/771.jpg
/kaggle/input/ai-of-god-v20/test/208.jpg
/kaggle/input/ai-of-god-v20/test/2628.jpg
/kaggle/input/ai-of-god-v20/test/820.jpg
/kaggle/input/ai-of-god-v20/test/1789.jpg
/kaggle/input/ai-of-god-v20/test/473.jpg
/kaggle/input/ai-of-god-v20/test/1792.jpg
/kaggle/input/ai-of-god-v20/test/2029.jpg
/kaggle/input/ai-of-god-v20/test/1411.jpg
/kaggle/input/ai-of-god-v20/test/1031.jpg
/kaggle/input/ai-of-god-v20/test/1833.jpg
/kaggle/input/ai-of-god-v20/test/2675.jpg

In [2]:
import pandas as pd
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image


In [3]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv('/kaggle/input/ai-of-god-v20/train.csv')
        self.root_dir = '/kaggle/input/ai-of-god-v20/train'
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data_frame.iloc[idx, 0])
        image = Image.open(img_name)
        label = int(self.data_frame.iloc[idx, 1])

        if self.transform:
            image = self.transform(image)

        return image, label


In [4]:
data_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.RandomCrop(size=(224, 224)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [5]:
csv_file = '/kaggle/input/ai-of-god-v20/train.csv'
root_dir = '/kaggle/input/ai-of-god-v20/train'

custom_dataset = CustomDataset(csv_file=csv_file, root_dir=root_dir, transform=data_transform)

batch_size = 32  

# Creating data loaders
train_size = int(0.9 * len(custom_dataset))
val_size = len(custom_dataset)-train_size

train_dataset, val_dataset = torch.utils.data.random_split(custom_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)


In [6]:
import torchvision.models as models
import torch.nn as nn
import torchvision.models as models
model = models.resnet152(pretrained=True)


num_classes = 8
model.fc = nn.Linear(model.fc.in_features, num_classes)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth
100%|██████████| 230M/230M [00:01<00:00, 230MB/s] 


In [7]:
import torch

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model = model.to(device)


In [8]:
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
model = model.to('cuda')

criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)  


In [9]:
scheduler = ReduceLROnPlateau(optimizer,mode ='min',patience =5,factor=0.1,verbose=True)

In [10]:
print(device)

cuda


In [11]:
def calculate_validation_loss(model, validation_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    num_samples = 0

    with torch.no_grad():
        for inputs, targets in validation_loader:
            inputs, targets = inputs.to(device), targets.to(device)  
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, targets)  # Calculating the loss
            total_loss += loss.item() * inputs.size(0)
            num_samples += inputs.size(0)

    model.train()  
    return total_loss / num_samples


In [ ]:
num_epochs = 30  # the number of training epochs 
i =0
for epoch in range(num_epochs):
    k =0
    print("Epoch =",i,"\n")
    model.train()  
    for inputs, labels in train_loader:
        print(k,"/",i)
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        k+=1

    i+=1
    print("=================================")
    model.eval()  

    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    validation_accuracy = correct / total
    print(f'Validation Accuracy: {validation_accuracy * 100:.2f}%')
    validation_loss = calculate_validation_loss(model, val_loader,criterion)
    scheduler.step(validation_loss)
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch {epoch}: Learning Rate = {current_lr}")
    if current_lr < 5e-8:
        break
    if validation_accuracy>=0.98:
        print("Accuracy reached")
        break
    print("=================================")



Epoch = 0 

0 / 0
1 / 0
2 / 0
3 / 0
4 / 0
5 / 0
6 / 0
7 / 0
8 / 0
9 / 0
10 / 0
11 / 0
12 / 0
13 / 0
14 / 0
15 / 0
16 / 0
17 / 0
18 / 0
19 / 0
20 / 0
21 / 0
22 / 0
23 / 0
24 / 0
25 / 0
26 / 0
27 / 0
28 / 0
29 / 0
30 / 0
31 / 0
32 / 0
33 / 0
34 / 0
35 / 0
36 / 0
37 / 0
38 / 0
39 / 0
40 / 0
41 / 0
42 / 0
43 / 0
44 / 0
45 / 0
46 / 0
47 / 0
48 / 0
49 / 0
50 / 0
51 / 0
52 / 0
53 / 0
54 / 0
55 / 0
56 / 0
57 / 0
58 / 0
59 / 0
60 / 0
61 / 0
62 / 0
63 / 0
64 / 0
65 / 0
66 / 0
67 / 0
68 / 0
69 / 0
70 / 0
71 / 0
72 / 0
73 / 0
74 / 0
75 / 0
76 / 0
77 / 0
78 / 0
79 / 0
80 / 0
81 / 0
82 / 0
83 / 0
84 / 0
85 / 0
86 / 0
87 / 0
88 / 0
89 / 0
90 / 0
91 / 0
92 / 0
93 / 0
94 / 0
95 / 0
96 / 0
97 / 0
98 / 0
99 / 0
100 / 0
101 / 0
102 / 0
103 / 0
104 / 0
105 / 0
106 / 0
107 / 0
108 / 0
109 / 0
110 / 0
111 / 0
112 / 0
113 / 0
114 / 0
115 / 0
116 / 0
117 / 0
118 / 0
119 / 0
120 / 0
121 / 0
122 / 0
123 / 0
124 / 0
125 / 0
126 / 0
127 / 0
128 / 0
129 / 0
130 / 0
131 / 0
132 / 0
133 / 0
134 / 0
135 / 0
136 / 0
13

In [ ]:
import os
import torch
from torchvision.datasets import DatasetFolder
from torchvision.transforms import transforms
from PIL import Image


In [ ]:
import os
import cv2
import torch
from torch.utils.data import Dataset
from torchvision.transforms import ToPILImage, transforms

class TestData(Dataset):
    def __init__(self, transform=None):
        self.dir = "/kaggle/input/ai-of-god-v20/test/"
        self.transform = transform
        self.to_pil = ToPILImage()  
        
    def __len__(self):
        return len(os.listdir(self.dir))
    
    def __getitem__(self, idx):
        img = cv2.imread(self.dir + str(idx + 1) + '.jpg')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.to_pil(img)  
        if self.transform is not None:
            img = self.transform(img)  
        return img


In [ ]:
test_dataset = TestData(transform=data_transform)

test_batch_size = 32  

test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)


In [ ]:
model.eval()  

predicted_labels = []  # List to store predicted labels
ll =0
with torch.no_grad():
    for inputs in test_loader:  
        print("imageno = ",ll)
        inputs = inputs.to(device)  
        outputs = model(inputs)  

        _, predicted = torch.max(outputs, 1)

        # Appending the batch of predicted labels to the list
        predicted_labels.extend(predicted.cpu().numpy())  # Convert to CPU and extract as NumPy array
        print("===============")
        ll+=1


In [ ]:
print(len(predicted_labels))

In [ ]:
import csv

csv_file = "my_data.csv"

with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(["class"])

    for value in predicted_labels:
        writer.writerow([value])

print(f"CSV file '{csv_file}' has been created.")
